In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ucl-datathon/sample_submission.csv
/kaggle/input/ucl-datathon/test_matchups.json
/kaggle/input/ucl-datathon/train.json
/kaggle/input/test-fake-v2/test_fake.json
/kaggle/input/european-soccer-data/Full_Dataset.csv


In [6]:
df = pd.read_csv("/kaggle/input/european-soccer-data/Full_Dataset.csv");
print(df['Competition'].unique())

['primera-division' 'premier-league' 'bundesliga' 'ligue-1' 'serie-a'
 'uefa-champions-league' 'uefa-europa-league' 'fa-cup']


uefa_df = df[df['Competition']=='uefa-champions-league']
uefa_df["Round"].unique()

We are given data of all matches of all the time, but we are interested in only knowing how the matches will unfold from ROUND16 to the finals.

We also donot need the first group stage, in the older format, ROUND 16, was essentially the second group stage and onward.

Day1 Day2 format is some weird thing recently introduced, it is same as league stage I believe

uefa_df_16 = uefa_df[~uefa_df['Round'].isin(['first-group-stage', 'group-stage'])]
uefa_df_16

This is the main Training and Testing data. To this, we can gather information from other matches. Create new columns and do something.

All these values are null, that means, we will not have any use of it, so drop!

print(uefa_df_16["Away_Score_AET"].isnull().mean()*100)
print(uefa_df_16["Home_Score_AET"].isnull().mean()*100)
print(uefa_df_16["Home_Penalties"].isnull().mean()*100)
print(uefa_df_16["Away_Penalties"].isnull().mean()*100)

In football too, there exists home and away advantage. In the original dataset, there is no mention of Home or away, so we can drop that column along with location, country, team points, opponent points, as all this info is not even there in the Test set

uefa_df_16 = uefa_df_16.drop(['Away_Score_AET', 'Home_Score_AET', 'Home_Penalties', 'Away_Penalties', 'Location', 'Team_Points', 'Opponent_Points', 'Competition', 'Country'], axis=1)


Remove the seasons as they are not in our training set

uefa_df_16 = uefa_df_16[~uefa_df_16['season'].isin([2002,2003])]

**Was it a wise idea to include only UCL data? Should not we include aggregate data from all leauges. But how do we provide greater weightage to UCL scores?**

uefa_df_16

Final Plan - 
* Divide the given Huge dataset into UEFA CL and non uefa league games, extract the details for each of these teams ONLY aggregate details.
* Then from this find attack strength, goal difference, defense strength. (all normalized)
* Weight each of them UCL gets 0.7 and other leagues get 0.3.
* Add these to the train and test dataset

We donot need to seperate into team and opponent, because the dataset has matches in a way such that if Arsenalvs Dortmund is mentioned then even the same match is written as Dortmund vs Arsenal

In [16]:
def calculate_team_stats_up_to_date(df, team, season, cutoff_date):
    """
    Calculate team stats only for matches played up to the cutoff date
    We do not need to do seperately for opponent and team, because of the nature of this dataset.
    We donot even need to consider home and away games due to the nature of the league.
    """
    df['Date'] = pd.to_datetime(df['Date'])
    cutoff_date = pd.to_datetime(cutoff_date)

    # Filter matches for this team and season, up to the cutoff date
    team_matches = df[
        (df['Team'] == team) & 
        (df['Date'] < cutoff_date) &
        (df['season'].between(season - 4, season))
    ].copy()
    if team_matches.empty:
        team = team.split()
        for word in team:
            team_matches = df[(df['Team'].str.contains(word, case=False, na=False)) &
            (df['season'].between(season - 4, season)) & (df['Date'] < cutoff_date)]
            if not team_matches.empty:
                break
    if team_matches.empty:
        return None

    matches_played = len(team_matches)
    goals_scored = team_matches['Team_Score'].sum()
    goals_conceded = team_matches['Opponent_Score'].sum()
    points = team_matches['Team_Points'].sum()
    
    goals_per_game = goals_scored / matches_played
    goals_conceded_per_game = goals_conceded / matches_played
    points_per_game = points / matches_played
    goal_difference = goals_scored - goals_conceded
    goal_diff_per_game = goal_difference / matches_played
    
    return {
        'matches_played': matches_played,
        'goals_scored': goals_scored,
        'goals_conceded': goals_conceded,
        'points': points,
        'goals_per_game': goals_per_game,
        'goals_conceded_per_game': goals_conceded_per_game,
        'points_per_game': points_per_game,
        'goal_difference': goal_difference,
        'goal_diff_per_game': goal_diff_per_game,
        'attack_strength': goals_per_game,
        'defense_strength': goals_conceded_per_game
    }
    
    return calculated_stats

In [8]:
def create_match_features(team1, team2, season, match_date, original_match_data):
    # Convert input team names to lowercase
    team1 = team1.lower()
    team2 = team2.lower()
    ucl_matches = original_match_data[original_match_data['Competition'] == 'uefa-champions-league'].copy()
    league_matches = original_match_data[original_match_data['Competition'] != 'uefa-champions-league'].copy()
    # Calculate UCL stats up to match date
    team1_ucl_stats = calculate_team_stats_up_to_date(ucl_matches, team1, season, match_date)
    team2_ucl_stats = calculate_team_stats_up_to_date(ucl_matches, team2, season, match_date)

    # Calculate League stats up to match date
    team1_league_stats = calculate_team_stats_up_to_date(league_matches, team1, season, match_date)
    team2_league_stats = calculate_team_stats_up_to_date(league_matches, team2, season, match_date)

    if not team1_ucl_stats and not team2_ucl_stats:
        # Use league stats only
        if not team1_league_stats or not team2_league_stats:
            return None

        def safe_divide(numerator, denominator):
            return numerator / denominator if denominator != 0 else np.nan
    
        features = {
                # Use league stats as primary
                'attack_strength_ratio': team1_league_stats['attack_strength'] / max(team2_league_stats['attack_strength'], 0.1),
                'ucl_attack_ratio': 1.0,  # No UCL data
                'league_attack_ratio': team1_league_stats['attack_strength'] / max(team2_league_stats['attack_strength'], 0.1),
                
                'defense_strength_ratio': max(team2_league_stats['defense_strength'], 0.1) / max(team1_league_stats['defense_strength'], 0.1),
                'ucl_defense_ratio': 1.0,  # No UCL data
                'league_defense_ratio': max(team2_league_stats['defense_strength'], 0.1) / max(team1_league_stats['defense_strength'], 0.1),
                
                'overall_form_ratio': team1_league_stats['points_per_game'] / max(team2_league_stats['points_per_game'], 0.1),
                'ucl_form_ratio': 1.0,  # No UCL data
                'league_form_ratio': team1_league_stats['points_per_game'] / max(team2_league_stats['points_per_game'], 0.1),
                
                'goal_diff_ratio': team1_league_stats['goal_diff_per_game'] - team2_league_stats['goal_diff_per_game'],
                'league_goal_diff_ratio': team1_league_stats['goal_diff_per_game'] - team2_league_stats['goal_diff_per_game'],
                'ucl_experience_diff': 0
            }
    else:
        if not team1_ucl_stats:
            team1_ucl_stats = {'attack_strength': 0, 'defense_strength': 1, 'points_per_game': 0, 'goal_diff_per_game': 0, 'matches_played': 0}
        if not team2_ucl_stats:
            team2_ucl_stats = {'attack_strength': 0, 'defense_strength': 1, 'points_per_game': 0, 'goal_diff_per_game': 0, 'matches_played': 0}
        if not team1_league_stats:
            team1_league_stats = {'attack_strength': 0, 'defense_strength': 1, 'points_per_game': 0, 'goal_diff_per_game': 0}
        if not team2_league_stats:
            team2_league_stats = {'attack_strength': 0, 'defense_strength': 1, 'points_per_game': 0, 'goal_diff_per_game': 0}
        
        # Calculate combined attack and defense strength (weighted)
        team1_total_attack = 0.6 * team1_ucl_stats['attack_strength'] + 0.4 * team1_league_stats['attack_strength']
        team2_total_attack = 0.6 * team2_ucl_stats['attack_strength'] + 0.4 * team2_league_stats['attack_strength']
        
        team1_total_defense = 0.6 * team1_ucl_stats['defense_strength'] + 0.4 * team1_league_stats['defense_strength']
        team2_total_defense = 0.6 * team2_ucl_stats['defense_strength'] + 0.4 * team2_league_stats['defense_strength']
        
        team1_overall_form = 0.7 * team1_ucl_stats['points_per_game'] + 0.3 * team1_league_stats['points_per_game']
        team2_overall_form = 0.7 * team2_ucl_stats['points_per_game'] + 0.3 * team2_league_stats['points_per_game']
        
        features = {
            # Attack ratios
            'attack_strength_ratio': team1_total_attack / max(team2_total_attack, 0.1),
            'ucl_attack_ratio': team1_ucl_stats['attack_strength'] / max(team2_ucl_stats['attack_strength'], 0.1),
            'league_attack_ratio': team1_league_stats['attack_strength'] / max(team2_league_stats['attack_strength'], 0.1),
            
            # Defense ratios (lower is better, so we flip)
            'defense_strength_ratio': max(team2_total_defense, 0.1) / max(team1_total_defense, 0.1),
            'ucl_defense_ratio': max(team2_ucl_stats['defense_strength'], 0.1) / max(team1_ucl_stats['defense_strength'], 0.1),
            'league_defense_ratio': max(team2_league_stats['defense_strength'], 0.1) / max(team1_league_stats['defense_strength'], 0.1),
            
            # Form ratios
            'overall_form_ratio': team1_overall_form / max(team2_overall_form, 0.1),
            'ucl_form_ratio': team1_ucl_stats['points_per_game'] / max(team2_ucl_stats['points_per_game'], 0.1),
            'league_form_ratio': team1_league_stats['points_per_game'] / max(team2_league_stats['points_per_game'], 0.1),
            
            # Goal difference ratios
            'goal_diff_ratio': team1_ucl_stats['goal_diff_per_game'] - team2_ucl_stats['goal_diff_per_game'],
            'league_goal_diff_ratio': team1_league_stats['goal_diff_per_game'] - team2_league_stats['goal_diff_per_game'],
            
            # Experience features
            'ucl_experience_diff': team1_ucl_stats['matches_played'] - team2_ucl_stats['matches_played']
        }

    return features



In [9]:
def add_match_features(fd, original_df):
    # Original_df has all the matches historically basically the EU Soccer data.
    feature_columns =  ['attack_strength_ratio', 'ucl_attack_ratio', 'league_attack_ratio',
                        'defense_strength_ratio', 'ucl_defense_ratio', 'league_defense_ratio', 
                        'overall_form_ratio', 'ucl_form_ratio', 'league_form_ratio',
                        'goal_diff_ratio', 'league_goal_diff_ratio', 'ucl_experience_diff']
    for col in feature_columns:
        fd[col] = None
    # Original_df is required when we are going to be using rolling stats.
    fd['date'] = pd.to_datetime(fd['date'])
    original_df['Date'] = pd.to_datetime(original_df['Date'], dayfirst=True)
    for idx, row in fd.iterrows():
        features = create_match_features(
            row['team1'], 
            row['team2'], 
            row['season'], 
            row['date'],  # Pass the match date
            original_df
        )

        if features:
            for feature_name, feature_value in features.items():
                fd.at[idx, feature_name] = feature_value
        else:
            print(f"Warning: Could not calculate features for {row['team1']} vs {row['team2']} in {row['season']}")

    return fd

In [10]:
import json
with open('/kaggle/input/ucl-datathon/train.json') as file:
    data = json.load(file)
matches = []

for season, season_data in data.items():
    for round_name, round_matches in season_data.items():
        for match_no, match_details in enumerate(round_matches, 1):
            match = {
                'season':season,
                'round': round_name.replace('_',' ').title(),
                'match_no': f"Match {match_no}",
                'team1': match_details.get('team_1'),
                'team2':match_details.get('team_2'),
                'date': match_details.get('date'),
                'winner':match_details.get('winner')
            }
            matches.append(match)

train_df = pd.DataFrame(matches)
# Standardize the names
train_df['team1'] = train_df['team1'].str.lower()
train_df['team2'] = train_df['team2'].str.lower()
train_df['winner'] = train_df['winner'].str.lower()

if 'date' in train_df.columns and not train_df['date'].isna().all():
    try:
        train_df['date'] = pd.to_datetime(train_df['date'], dayfirst=True)
    except:
        pass

In [11]:
def get_result(row):
    if row['winner'] == row['team1']:
        return 'team1_win'
    elif row['winner'] == row['team2']:
        return 'team2_win'
    else:
        return 'draw'
        
def create_target_variables(fd):
    fd['team1_wins'] = fd.apply(lambda row: 1 if row['winner'] == row['team1']
                               else (0 if row['winner'] == row['team2'] else 0.5), axis=1)
    fd['result'] = fd.apply(get_result, axis=1)
    return fd

In [12]:
def create_ucl_training_dataframe(fd):
    fd = add_match_features(fd, df.copy())
    fd = create_target_variables(fd)
    return fd

In [13]:
train_df['season']=train_df['season'].str.split('-').str[0].astype(int)
train_df['team1'] = train_df['team1'].replace('internazionale', 'inter')
train_df['team2'] = train_df['team2'].replace('internazionale', 'inter')
train_df['date']

0     2005-02-22
1     2005-02-22
2     2005-02-22
3     2005-02-22
4     2005-02-23
         ...    
190   2017-04-12
191   2017-04-12
192   2017-05-02
193   2017-05-03
194   2017-06-03
Name: date, Length: 195, dtype: datetime64[ns]

In [17]:
train_df = create_ucl_training_dataframe(train_df)

In [18]:
train_df

,season,round,match_no,team1,team2,date,winner,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,team1_wins,result
0,2004,Round Of 16,Match 1,real madrid,juventus,2005-02-22,juventus,1.004574,1.085,0.895652,...,0.708412,0.806789,0.911747,0.961574,0.809993,-0.234677,-0.428214,9,0.0,team2_win
1,2004,Round Of 16,Match 2,liverpool,bayer leverkusen,2005-02-22,liverpool,0.890468,1.036364,0.732473,...,1.665072,1.147592,1.145861,1.357143,0.855255,0.662281,-0.386088,-26,1.0,team1_win
2,2004,Round Of 16,Match 3,psv eindhoven,monaco,2005-02-22,psv eindhoven,0.346064,0.542042,0.0,...,0.990431,0.82243,0.570047,0.83165,0.0,-0.903509,-0.831776,-1,1.0,team1_win
3,2004,Round Of 16,Match 4,bayern munich,arsenal,2005-02-22,bayern munich,0.963924,0.933333,0.995527,...,0.742857,0.987582,0.796798,0.7,0.956043,-0.421429,-0.021464,-8,1.0,team1_win
4,2004,Round Of 16,Match 5,barcelona,chelsea,2005-02-23,chelsea,1.113462,1.246154,0.954508,...,1.031746,0.749451,1.087919,1.182857,0.878911,0.377778,-0.335193,2,0.0,team2_win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2016,Quarter Finals,Match 3,bayern munich,real madrid,2017-04-12,real madrid,1.226554,1.130885,1.393078,...,1.033761,2.086215,1.153083,1.046252,1.439163,0.317434,1.315771,-7,0.0,team2_win
191,2016,Quarter Finals,Match 4,atletico madrid,leicester city,2017-04-12,atletico madrid,1.24811,1.254545,1.239543,...,1.833333,1.787209,1.108253,1.011364,1.416536,0.772727,0.920977,36,1.0,team1_win
192,2016,Semi Finals,Match 1,real madrid,atletico madrid,2017-05-02,real madrid,1.255087,1.393939,1.071833,...,0.527494,0.653747,0.9677,1.01173,0.867864,0.121212,-0.262244,20,1.0,team1_win
193,2016,Semi Finals,Match 2,monaco,juventus,2017-05-03,juventus,0.937857,0.94,0.935388,...,0.729483,0.650645,0.922979,0.979167,0.825423,-0.373404,-0.466529,-27,0.0,team2_win


In [19]:
train_df.to_csv('train_data_with_feature_addn.csv', index=False)

In [ ]:
import json
import pandas as pd
with open('/kaggle/input/test-fake-v2/test_fake.json') as file:
    data = json.load(file)
matches = []

for season, season_data in data.items():
    for round_name, round_matches in season_data.items():
        if isinstance(round_matches, dict):
            # Single match (e.g., final_matchup)
            match = {
                'season': season,
                'round': round_name.replace('_', ' ').title(),
                'match_no': "Match 1",
                'team1': round_matches.get('team_1'),
                'team2': round_matches.get('team_2'),
                'date': round_matches.get('date')
            }
            matches.append(match)
        elif isinstance(round_matches, list):
            # Multiple matches
            for match_no, match_details in enumerate(round_matches, 1):
                match = {
                    'season': season,
                    'round': round_name.replace('_', ' ').title(),
                    'match_no': f"Match {match_no}",
                    'team1': match_details.get('team_1'),
                    'team2': match_details.get('team_2'),
                    'date': match_details.get('date')
                }
        
                matches.append(match)

test_df = pd.DataFrame(matches)
# Standardize the names
test_df['team1'] = test_df['team1'].str.lower()
test_df['team2'] = test_df['team2'].str.lower()

if 'date' in test_df.columns and not test_df['date'].isna().all():
    try:
        test_df['date'] = pd.to_datetime(test_df['date'], dayfirst=False)
    except:
        pass

In [ ]:
def create_ucl_testing_dataframe(fd):
    fd = add_match_features(fd, df.copy())
    return fd

In [ ]:
test_df['season']=test_df['season'].str.split('-').str[0].astype(int)
test_df['team1'] = test_df['team1'].replace('internazionale', 'inter')
test_df['team2'] = test_df['team2'].replace('internazionale', 'inter')
test_df['date']

In [ ]:
test_df = create_ucl_testing_dataframe(test_df)

In [ ]:
test_df.to_csv('test_data_with_feature_addn.csv', index=False)